In [ ]:
import json
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

# 1. 임베딩 모델 설정
e_model_id = "text-embedding-3-small"
embedding_model = OpenAIEmbeddings(model=e_model_id)

# 2. Chroma DB 로드
db = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embedding_model
)

# 3. 카드사 리스트
brands = ["신한", "하나"]

# 4. 각 카드사 데이터 처리
for brand in brands:
    with open(f"../data/raw/{brand}.json", "r", encoding="utf-8") as f:
        card_data = json.load(f)

    documents = []
    for card in card_data:
        doc_text = f"{card['name']}는 {card['brand']}에서 발급한 {card['c_brand']} 카드입니다. "
        doc_text += f"연회비는 국내 {card['fee_domestic']}원, 해외겸용 {card['fee_global']}원입니다.\n"

        for b in card['benefits']:
            doc_text += f"- [{b['category']}] {b['short_description']} / {b['detail_description']}\n"

        documents.append(
            Document(page_content=doc_text, metadata={"name": card['name'], "url": card['url']})
        )

    db.add_documents(documents)
    print(f"✅ {brand}카드 {len(documents)}건 임베딩 추가 완료")

# 5. DB 저장
db.persist()
print("✅ 모든 카드 문서가 Chroma DB에 저장되었습니다.")


C:\Users\Playdata\AppData\Local\Temp\ipykernel_20408\1811007661.py:14: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


✅ 신한카드 10건 임베딩 추가 완료
✅ 하나카드 10건 임베딩 추가 완료
✅ 모든 카드 문서가 Chroma DB에 저장되었습니다.


C:\Users\Playdata\AppData\Local\Temp\ipykernel_20408\1811007661.py:43: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [7]:
coll = db._collection

In [ ]:
coll.count() # 저장된 데이터개수

20

In [9]:
coll.get()

{'ids': ['f09034b1-31f9-44da-8415-d82ad0925df3',
  '0974d6b6-09e2-4fe5-b47b-ae609d60aacd',
  'd4929fb3-f115-4ce8-82f2-7f5e70a04ed9',
  '99a57104-8e1a-4836-a976-9e582567d1d2',
  'f0ae03f3-43d5-4fbf-985a-4a0a1483f0c1',
  'c15c4d82-f840-44af-a98b-9ec676531b3d',
  '54948f2d-38bf-4d6d-9802-51cc7a00e68c',
  '150eb31b-024a-4df0-bb3a-297de595701d',
  '4d68b0e7-f27c-41f9-ac24-6982ff1f67da',
  '4bb6f89d-01e5-49a1-9a8e-0fefc6b8466d',
  '6c0b97e0-0363-46fc-aca3-dbdecb795630',
  'accd3fa3-c85f-4263-a520-45844d1a1308',
  '6be1c100-53ee-4def-9b1a-1fe931317314',
  'a7b55ecc-5d41-4f25-95df-74dda36b084b',
  'f5556fcb-59f0-4382-b4f8-b246cfa2fcb1',
  '94b7b59e-2534-4a6a-bbda-66b15c8416ad',
  '22e9744e-f1dd-4ef3-af8c-831d51e34a4f',
  'fbe66c24-c334-4f01-a13d-b90d74bba66a',
  '261f23c9-2bbe-45db-8fb7-93dc91cf7200',
  '5b1dbd00-eed9-445c-8891-aec86874a1cc'],
 'embeddings': None,
 'documents': ['신한카드 처음(ANNIVERSE)는 신한카드에서 발급한 Mastercard 카드입니다. 연회비는 국내 15,000원, 해외겸용 18,000원입니다.\n- [적립] [오늘도 5% 적립 서비스] 음식점·카페·편

In [6]:
db.similarity_search("교통 할인 카드")

[Document(metadata={'url': 'https://www.card-gorilla.com/card/detail/13', 'name': '신한카드 Mr.Life'}, page_content='신한카드 Mr.Life는 신한카드에서 발급한 VISA 카드입니다. 연회비는 국내 원, 해외겸용 15,000원입니다.\n- [공과금] 월납요금(공과금) 10% 할인서비스 / 월납요금(공과금) 10% 할인서비스- 전기요금 /도시가스요금/ SKT, LG U+, KT 통신요금 할인* 인터넷/집전화/이동통신/결합상품 포함- 일 1회 할인 적용- 1회 이용 금액 5만원까지 할인 적용 (1회 최대 5천원 할인)할인한도안내- 전월실적 30만원~50만원 : 3천원- 전월실적 50만원 ~100만원 : 7천원- 전월실적 100만원 이상 : 1만원- 서비스 대상 거래건 중 신한카드 전표 매입 순서대로 결제일 할인이 적용됩니다.- 월납(공과금) 할인서비스는 전월 이용금액에 따라 제공된 월 할인 한도 내에서 서비스가 제공됩니다.- 신규 발급 회원에 대해서는 카드사용 등록월의 익월말(등록월+1개월)까지 3천원의 할인한도가 제공됩니다.- 전월 이용실적은 일시불+할부 금액 기준이며 (전월 할인거래 실적 포함) 교통이용금액은 전전월 이용금액 기준, 해외이용- 금액은 매입일자를 기준으로 적용됩니다- 월 기준 : 매월 1일 ~ 말일Powered byFroala Editor\n- [편의점] 편의점 10% 할인 / TIME 할인서비스 10% 할인편의점 10% 할인- 편의점 업종- 서비스 영역별 각각 일 1회 할인 적용- 1회 이용 금액 1만원까지 할인 적용 (1회 최대 1천원 할인)- 월 5회 할인 적용TIME 할인서비스 할인한도안내- 전월실적 30만원~50만원 : 1만원- 전월실적 50만원 ~100만원 : 2만원- 전월실적 100만원 이상 : 3만원유의사항- 서비스 대상 거래건 중 신한카드 전표 매입 순서대로 결제일 할인이 적용됩니다.- TIME 할인서비스는 전월 이용금액에 따라 제공된 월 할인 한

In [2]:
from langchain.vectorstores import Chroma

retriever = Chroma(
    embedding_function=embedding_model,
    persist_directory="./chroma_db"
).as_retriever()

In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0) 

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)


C:\Users\Playdata\AppData\Local\Temp\ipykernel_20408\1936542503.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)


In [4]:
from langgraph.graph import StateGraph, END
from typing import TypedDict


# 상태 정의
class CardState(TypedDict):
    question: str
    answer: str
    sources: list

# 노드 1: 유저 질문 받아 저장
def get_question(state):
    return state

# 노드 2: RAG 체인으로 답변 생성
def generate_answer(state):
    result = rag_chain({"query": state["question"]})
    return {
        "question": state["question"],
        "answer": result["result"],
        "sources": result.get("source_documents", [])
    }

# LangGraph 정의
graph = StateGraph(CardState)
graph.add_node("input", get_question)
graph.add_node("rag", generate_answer)

graph.set_entry_point("input")
graph.add_edge("input", "rag")
graph.set_finish_point("rag")

app = graph.compile()


In [5]:
result = app.invoke({"question": "대중교통, 편의점, 그리고 커피 할인 모두 되는 카드 있어?"})
print("답변:", result["answer"])
print("출처:", [doc.metadata for doc in result["sources"]])

C:\Users\Playdata\AppData\Local\Temp\ipykernel_20408\3998200377.py:17: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = rag_chain({"query": state["question"]})


답변: 대중교통, 편의점, 그리고 커피 할인 모두 가능한 카드는 다음과 같습니다.

1. 원더카드 (원더 Life, 하나카드)
- 대중교통 10% 할인 (버스, 지하철)
- 편의점 10% 할인 (CU, GS25, 세븐일레븐, 이마트24)
- 커피 10% 할인 (스타벅스, 커피빈, 이디야, 폴바셋, 투썸플레이스, 블루보틀)

2. K-패스 신한카드 (신한카드)
- 대중교통 10% 결제일 할인 (버스, 지하철)
- 편의점 5% 결제일 할인 (GS25, CU)
- 커피전문점 5% 결제일 할인 (스타벅스, 메가MGC커피, 매머드커피)

3. 신한카드 Mr.Life (신한카드)
- 대중교통 할인 명시 없음 (따로 대중교통 할인은 없으나 택시 10% 할인 있음)
- 편의점 10% 할인 (TIME 할인서비스)
- 커피는 식음료 10% 할인에 포함 가능 (한식, 양식, 일식, 중식, 뷔페, 일반대중음식, 패스트푸드, 커피전문점 업종)

따라서 대중교통, 편의점, 커피 할인 모두를 원하시면 원더카드(원더 Life)와 K-패스 신한카드가 적합하며, 할인율은 원더카드가 더 높습니다.
출처: [{'url': 'https://www.card-gorilla.com/card/detail/2654', 'name': '원더카드 (원더 Life)'}, {'name': 'K-패스 신한카드', 'url': 'https://www.card-gorilla.com/card/detail/2690'}, {'name': '원더카드 (원더 HAPPY+)', 'url': 'https://www.card-gorilla.com/card/detail/2821'}, {'name': '신한카드 Mr.Life', 'url': 'https://www.card-gorilla.com/card/detail/13'}]


In [6]:
result = app.invoke({"question": "그러면 k-패스 신한카드 정보에 대해 더 알려줘봐."})
print("답변:", result["answer"])

답변: K-패스 신한카드는 신한카드에서 발급하는 Mastercard 카드로, 주요 혜택은 다음과 같습니다:

- 연회비: 국내용 7,000원, 해외겸용 10,000원입니다.

- 대중교통 할인: 버스와 지하철 이용 시 10% 결제일 할인이 적용됩니다. 후불교통 기능을 이용한 터치(RF) 거래에 한하며, 고속버스, 공항버스, 시외버스, 공항철도는 할인 대상에서 제외됩니다. 할인은 카드 사용일이 아닌 이용대금명세서에 기재된 이용일 기준으로 제공됩니다.

- 간편결제 할인: 신한 SOL페이, 삼성페이, 네이버페이, 카카오페이, 쿠페이 등 간편결제에 K-패스 신한카드를 등록 후 국내 온/오프라인 결제 시 5% 결제일 할인이 적용됩니다. 단, 해외 거래 및 교통카드 이용금액은 할인 제외입니다.

- 생활서비스 할인: 배달앱(배달의민족, 요기요)에서 앱 내 카드결제 시 5% 할인, 편의점(GS25, CU) 오프라인 매장 결제 시 5% 할인, 커피전문점(스타벅스, 메가MGC커피, 매머드커피) 오프라인 및 공식 앱 결제 시 5% 할인, 올리브영 오프라인 매장 및 공식 홈페이지/앱 결제 시 5% 할인, 병원/약국 업종 5% 할인(동물병원 제외), OTT(넷플릭스, 유튜브 프리미엄) 정기결제 시 5% 할인, 이동통신요금(SK텔레콤, KT, LG U+) 자동이체 시 5% 할인 등이 제공됩니다.

- 이동통신요금 할인은 월 1회 적용되며, 약정금액 대납, 지점 월납, 알뜰폰 요금은 할인 제외입니다.

이 카드의 할인 혜택은 결제일 할인 방식으로 제공되며, 각 할인 항목별로 조건과 제외 대상이 있으니 자세한 내용은 신한카드 홈페이지나 고객센터를 통해 확인하는 것이 좋습니다.
